# Principal Component Analysis

[data source](https://archive.ics.uci.edu/ml/datasets/Wine)
- label: 3
- feature: 13
- number of samples: 178

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import plotly.express as px
import plotly.graph_objects as go
import plotly
plotly.offline.init_notebook_mode(connected=True)
import pandas as pd
# import matplotlib.pyplot as plt
# %matplotlib inline
import numpy as np

df = pd.read_csv('https://raw.githubusercontent.com/rasbt/python-machine-learning-book-2nd-edition/master/code/ch05/wine.data', header=None)

In [2]:
columns = ['target']
columns += [f'feature_{i}' for i in range(1, len(df.columns))]
df.columns = columns

In [3]:
df.head()

,target,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


In [4]:
# label -> first column
X, y = df.iloc[:, 1:].values, df.iloc[:, 0].values

# (178 samples x 13 features)
print(X.shape)
X[0]

(178, 13)


array([1.423e+01, 1.710e+00, 2.430e+00, 1.560e+01, 1.270e+02, 2.800e+00,
       3.060e+00, 2.800e-01, 2.290e+00, 5.640e+00, 1.040e+00, 3.920e+00,
       1.065e+03])

In [5]:
# visualize the first 3 features
data = go.Scatter3d(
    x=X[:, 0],
    y=X[:, 1],
    z=X[:, 2],
    mode='markers',
    marker=dict(size=4, color=y)
)
layout = go.Layout(title='First 3 features of original data')
fig = go.Figure(data=data, layout=layout)
fig.show()

In [6]:
# Stadardize data 
sc = StandardScaler()
X_std = sc.fit_transform(X)

In [7]:
# check label
set(y)

{1, 2, 3}

In [8]:
# manually caluculate "covariance" from variance
S = np.dot((X_std - np.mean(X_std, axis=0)).T, (X_std - np.mean(X_std, axis=0))) / (X_std.shape[0] - 1)
print(S.shape)
print(S[:5])

(13, 13)
[[ 1.00564972  0.09493026  0.21273976 -0.31198788  0.27232816  0.29073446
   0.23815287 -0.15681042  0.13747022  0.549451   -0.07215255  0.07275191
   0.64735687]
 [ 0.09493026  1.00564972  0.16497228  0.29013035 -0.05488343 -0.3370606
  -0.41332866  0.29463237 -0.22199334  0.25039204 -0.56446685 -0.37079354
  -0.19309537]
 [ 0.21273976  0.16497228  1.00564972  0.44587209  0.28820583  0.12970824
   0.11572743  0.1872826   0.00970647  0.2603499  -0.07508874  0.00393333
   0.22488969]
 [-0.31198788  0.29013035  0.44587209  1.00564972 -0.0838039  -0.32292752
  -0.353355    0.36396647 -0.19844168  0.01883781 -0.27550299 -0.27833221
  -0.44308618]
 [ 0.27232816 -0.05488343  0.28820583 -0.0838039   1.00564972  0.21561254
   0.19688989 -0.25774204  0.23777643  0.20107967  0.05571118  0.06637684
   0.39557317]]


In [9]:
# Default: `rowvar=True` -> each row represents a variable, with observations(samples) in the columns.
# In this case, row represents observations.
# Check if the result is the same as the output above

cov_mat = np.cov(X_std, rowvar=False)
print(cov_mat.shape)
print(cov_mat[:5])

(13, 13)
[[ 1.00564972  0.09493026  0.21273976 -0.31198788  0.27232816  0.29073446
   0.23815287 -0.15681042  0.13747022  0.549451   -0.07215255  0.07275191
   0.64735687]
 [ 0.09493026  1.00564972  0.16497228  0.29013035 -0.05488343 -0.3370606
  -0.41332866  0.29463237 -0.22199334  0.25039204 -0.56446685 -0.37079354
  -0.19309537]
 [ 0.21273976  0.16497228  1.00564972  0.44587209  0.28820583  0.12970824
   0.11572743  0.1872826   0.00970647  0.2603499  -0.07508874  0.00393333
   0.22488969]
 [-0.31198788  0.29013035  0.44587209  1.00564972 -0.0838039  -0.32292752
  -0.353355    0.36396647 -0.19844168  0.01883781 -0.27550299 -0.27833221
  -0.44308618]
 [ 0.27232816 -0.05488343  0.28820583 -0.0838039   1.00564972  0.21561254
   0.19688989 -0.25774204  0.23777643  0.20107967  0.05571118  0.06637684
   0.39557317]]


In [10]:
# Compute eigendecompositon of covariance
W, V_pca = np.linalg.eig(cov_mat)
print(f'Eigen values \n{W}')

Eigen values 
[4.73243698 2.51108093 1.45424187 0.92416587 0.85804868 0.64528221
 0.55414147 0.10396199 0.35046627 0.16972374 0.29051203 0.22706428
 0.25232001]


In [11]:
V_pca.shape

(13, 13)

In [12]:
total = sum(W)
# cumlative sum of variance explain ratio
var_exp = [(i/total)*100 for i in sorted(W, reverse=True)]
var_exp_cum = np.cumsum(var_exp)
var_exp_cum

array([ 36.1988481 ,  55.40633836,  66.52996889,  73.59899908,
        80.16229276,  85.09811607,  89.3367954 ,  92.01754435,
        94.23969775,  96.16971684,  97.90655253,  99.20478511,
       100.        ])

14個の特徴量の内, 5個で80％の説明率に達していることがわかる.<br>
３軸で可視化できるよう, 今回は3次元に圧縮する

In [13]:
index = W.argsort()[::-1]
print(index)
V_pca = V_pca[:, index]

[ 0  1  2  3  4  5  6  8 10 12 11  9  7]


In [14]:
# transform data from 13 dimensions to 3 dimensions
X_pca = np.dot(X_std, V_pca[:, :3])
X_pca.shape

(178, 3)

In [15]:
# Visualize the result
data = go.Scatter3d(
    x=X_pca[:, 0],
    y=X_pca[:, 1],
    z=X_pca[:, 2],
    mode='markers',
    marker=dict(size=4, color=y)
)
layout = go.Layout(title='After PCA')
fig = go.Figure(data=data, layout=layout)
fig.show()

# Singular Value Decomposition

Let's see a singular value decomposition.

`np.linalg.svd`returns singular vectors and singular values in **descending order**. So you dont have to sort the order.

> s : (..., K) array
    Vector(s) with the singular values, within each vector sorted in
    descending order. The first ``a.ndim - 2`` dimensions have the same
    size as those of the input `a`.

In [16]:
U, S, V_T_svd = np.linalg.svd(X_std)
V_svd = V_T_svd.T
print(U.shape)
print(V_svd.shape)

(178, 178)
(13, 13)


In [17]:
V_svd[:5]

array([[-0.1443294 ,  0.48365155, -0.20738262,  0.0178563 , -0.26566365,
         0.21353865, -0.05639636,  0.39613926, -0.50861912,  0.21160473,
         0.22591696, -0.26628645,  0.01496997],
       [ 0.24518758,  0.22493093,  0.08901289, -0.53689028,  0.03521363,
         0.53681385,  0.42052391,  0.06582674,  0.07528304, -0.30907994,
        -0.07648554,  0.12169604,  0.02596375],
       [ 0.00205106,  0.31606881,  0.6262239 ,  0.21417556, -0.14302547,
         0.15447466, -0.14917061, -0.17026002,  0.30769445, -0.02712539,
         0.49869142, -0.04962237, -0.14121803],
       [ 0.23932041, -0.0105905 ,  0.61208035, -0.06085941,  0.06610294,
        -0.10082451, -0.28696914,  0.42797018, -0.20044931,  0.05279942,
        -0.47931378, -0.05574287,  0.09168285],
       [-0.14199204,  0.299634  ,  0.13075693,  0.35179658,  0.72704851,
         0.03814394,  0.3228833 , -0.15636143, -0.27140257,  0.06787022,
        -0.07128891,  0.06222011,  0.05677422]])

In [18]:
V_T_svd[:5]

array([[-0.1443294 ,  0.24518758,  0.00205106,  0.23932041, -0.14199204,
        -0.39466085, -0.4229343 ,  0.2985331 , -0.31342949,  0.0886167 ,
        -0.29671456, -0.37616741, -0.28675223],
       [ 0.48365155,  0.22493093,  0.31606881, -0.0105905 ,  0.299634  ,
         0.06503951, -0.00335981,  0.02877949,  0.03930172,  0.52999567,
        -0.27923515, -0.16449619,  0.36490283],
       [-0.20738262,  0.08901289,  0.6262239 ,  0.61208035,  0.13075693,
         0.14617896,  0.1506819 ,  0.17036816,  0.14945431, -0.13730621,
         0.08522192,  0.16600459, -0.12674592],
       [ 0.0178563 , -0.53689028,  0.21417556, -0.06085941,  0.35179658,
        -0.19806835, -0.15229479,  0.20330102, -0.39905653, -0.06592568,
         0.42777141, -0.18412074,  0.23207086],
       [-0.26566365,  0.03521363, -0.14302547,  0.06610294,  0.72704851,
        -0.14931841, -0.10902584, -0.50070298,  0.13685982, -0.07643678,
        -0.17361452, -0.10116099, -0.1578688 ]])

次元圧縮するときは、右特異ベクトルをそのまま使う(V_T_svd)のではなく、転置してから(V_svd)列数を制限することを忘れない

In [19]:
# transform data from 13 dimensions to 3 dimensions
X_svd = np.dot(X_std,V_svd[:, :3])

In [20]:
# Visualize the resultt
data = go.Scatter3d(
    x=X_svd[:, 0],
    y=X_svd[:, 1],
    z=X_svd[:, 2],
    mode='markers',
    marker=dict(size=4, color=y)
)
layout = go.Layout(
    title='After SVD'
)
fig = go.Figure(data=data, layout=layout)
fig.show()

PCAでの結果とほぼ一緒であることが伺える

## Using scikit-learn

In [21]:
from sklearn.decomposition import PCA, TruncatedSVD

In [22]:
pca = PCA(n_components=3, random_state=1)
svd = TruncatedSVD(n_components=3, random_state=1)
pca_ = pca.fit(X_std)
svd_ = svd.fit(X_std)
print('--------PCA---------')
print(sorted(pca_.singular_values_, reverse=True))
print('--------SCD---------')
print(sorted(svd_.singular_values_,reverse=True))

--------PCA---------
[28.942034224157347, 21.082251410776475, 16.043715611067917]
--------SCD---------
[28.942034224157354, 21.082251410776507, 16.04371561106791]


一発

## Without scaling

In [23]:
svd_nosc =svd.fit(X)
print('--------SCD---------')
print(sorted(svd_nosc.singular_values_,reverse=True))

--------SCD---------
[10886.669906564, 493.5620476385899, 57.14884322515755]


In [24]:
# trainデータを13次元から3次元に変換
X_svd_nosc = svd_nosc.transform(X_std)

In [25]:
data = go.Scatter3d(
    x=X_svd_nosc[:, 0],
    y=X_svd_nosc[:, 1],
    z=X_svd_nosc[:, 2],
    mode='markers',
    marker=dict(size=4, color=y)
)
layout = go.Layout(
    title='SVD without scaling'
)
fig = go.Figure(data=data, layout=layout)
fig.show()

うまく分類できてないことがわかる。いつも標準化した方が良さそう

## Experiment
右特異ベクトルVは転置する必要があるのか

In [26]:
# 特異値分解したままの状態で次元圧縮をしてみる
X_svd_T = np.dot(X_std, V_T_svd[:, :3])
V_T_svd == V_T_svd.T

array([[ True, False, False, False, False, False, False, False, False,
        False, False, False, False],
       [False,  True, False, False, False, False, False, False, False,
        False, False, False, False],
       [False, False,  True, False, False, False, False, False, False,
        False, False, False, False],
       [False, False, False,  True, False, False, False, False, False,
        False, False, False, False],
       [False, False, False, False,  True, False, False, False, False,
        False, False, False, False],
       [False, False, False, False, False,  True, False, False, False,
        False, False, False, False],
       [False, False, False, False, False, False,  True, False, False,
        False, False, False, False],
       [False, False, False, False, False, False, False,  True, False,
        False, False, False, False],
       [False, False, False, False, False, False, False, False,  True,
        False, False, False, False],
       [False, False, False,

転置にはなってなさそう

In [27]:
X_svd_T == X_svd

array([[False, False, False],
       [False, False, False],
       [False, False, False],
       [False, False, False],
       [False, False, False],
       [False, False, False],
       [False, False, False],
       [False, False, False],
       [False, False, False],
       [False, False, False],
       [False, False, False],
       [False, False, False],
       [False, False, False],
       [False, False, False],
       [False, False, False],
       [False, False, False],
       [False, False, False],
       [False, False, False],
       [False, False, False],
       [False, False, False],
       [False, False, False],
       [False, False, False],
       [False, False, False],
       [False, False, False],
       [False, False, False],
       [False, False, False],
       [False, False, False],
       [False, False, False],
       [False, False, False],
       [False, False, False],
       [False, False, False],
       [False, False, False],
       [False, False, False],
       [Fa

In [28]:
data = go.Scatter3d(
    x=X_svd_T[:, 0],
    y=X_svd_T[:, 1],
    z=X_svd_T[:, 2],
    mode='markers',
    marker=dict(size=4, color=y)
)
layout = go.Layout(
    title='After SVD witout transposing right singular vectors'
)
fig = go.Figure(data=data, layout=layout)
fig.show()

X_svdの時と比べてうまく分類できていない？